# Hybrid recommender
I did this one, just because I got intrested in how it works, so I thought, I'd try to make one myself.

## Plan:
* Load and merge the data on WineID. Analyze features using ydata profiling.
* Load the data, format columns to the correct types like:
    * Grapes and Harmonize (string list) -> (python list)  
    !!! This step is required in future always when loading this columns from csv.
    * Vintage (str) -> numeric. I just simply replace N.V.(non-vintage) with 0 and then turn whole column to integer type.
    * Datetime to a proper pd.datetime type
* Filter datasets to keep only wines and users with >=5 ratings.
* Merge datasets on WineID. Final size is 21,013,536 rows x 15 columns
* Columns used:
    * **WineID**: Integer. The wine primary key identification;
    * **WineName**: String. The textual wine identification presented in the label;
    * **Type**: String. The categorical type classification: Red, white or rosé for still wines, gasified sparkling or dessert for sweeter and fortified wines. Dessert/Port is a subclassification for liqueur dessert wines;
    * **Elaborate**: String. Categorical classification between varietal or assemblage/blend. The most famous blends are also considered, such as * Bordeaux red and white blend, Valpolicella blend and Portuguese red and white blend;
    * **Grapes**: String list. It contains the grape varieties used in the wine elaboration. The original names found have been kept;
    * **Harmonize**: String list. It contains the main dishes set that pair with the wine item. These are provided by producers but openly recommended on the internet by sommeliers and even consumers;
    * **ABV**: Float. The alcohol by volume (ABV) percentage. According to [1], the value shown on the label may vary, and a tolerance of 0.5% per 100 volume is allowed, reaching 0.8% for some wines;
    * **Body**: String. The categorical body classification: Very light-bodied, light-bodied, medium-bodied, full-bodied or very full-bodied based on wine viscosity [37];
    * **Acidity**: String. The categorical acidity classification: Low, medium, or high, based on potential hydrogen (pH) score [38];
    * **Country**: String. The categorical origin country identification of the wine production (ISO-3166);
    * **RegionName**: String. The textual wine region identification. The appellation region name was retained when identified;
    * **WineryName**: String. The textual winery identification;
    * **UserID**: Integer. The sequential key without identifying the user's private data;
    * **Vintage**: String. A rated vintage year or the abbreviation "N.V." referring to "non-vintage";
    * **Date**: String. Datetime in the format YYYY-MM-DD hh:mm:ss informing when it was rated by the user. It can be easily converted to other formats.
    * **Rating**(**Target variable**): Float. It contains the 5-stars (1–5) rating value ⊂ {1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5} performed by the user;
* Columns dropped:
    * **RegionID** and **RatingID** - since they are just unique IDs and not descriptive for the recommender. We care only about users IDs and wines IDs.
    * **Code** - since it's the same meaning as **Country** column. Either one can be selected.
    * **Vintages** - since it's just lists of possible vintages and we already have a Vintage column with the exact value(year or 0 for non-vintage).

#### Used ChatGPT for speeding up the process, since we have a large dataset, and also for beautifying outputs.

In [ ]:

import pandas as pd
import ast
import numpy as np
from ydata_profiling import ProfileReport
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from category_encoders import TargetEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import hstack, csr_matrix, issparse, lil_matrix, save_npz
import optuna


In [ ]:
# Converter of string lists into Python lists
# (e.g. "['a', 'b', 'c']" → [a, b, c])
def parse_list_col(s):
    return ast.literal_eval(s)

# Converter of 'N.V.' to 0, so column is numeric
def parse_vintage(s):
    return 0 if s == 'N.V.' else int(s)

# Time-based split of a DataFrame by user, could be replaced later with other method
# (e.g. 80% train, 20% test) based on the date column
def user_time_split(df, date_col='Date', split_ratio=0.8):
    train_parts, val_parts = [], []
    for uid, group in df.groupby('UserID', sort=False):
        group = group.sort_values(date_col)
        i = int(len(group) * split_ratio)
        train_parts.append(group.iloc[:i])
        val_parts.append(group.iloc[i:])
    return pd.concat(train_parts), pd.concat(val_parts)

In [ ]:
# Read the wines data and parse string lists into Python lists
print("▶ Reading wines metadata…")
wines = pd.read_csv(
    './data/XWines_Full_100K_wines.csv',
    usecols=['WineID', 'Type', 'Elaborate', 'ABV', 'Body', 'Acidity', 'RegionName', 'WineryName', 'Grapes','Harmonize','Country'],
    converters={
        'Grapes':    parse_list_col,
        'Harmonize': parse_list_col
    }
)
print(f"   ✓ Loaded wines: {len(wines):,} rows")

# Read the ratings data and parse 'N.V.' into 0 (Vitage to numeric), parse dates
print("▶ Reading ratings…")
ratings = pd.read_csv(
    './data/XWines_Full_21M_ratings.csv',
    usecols=['UserID','WineID','Date','Vintage','Rating'],
    parse_dates=['Date'],
    date_format=lambda s: pd.to_datetime(s),
    converters={'Vintage': parse_vintage}
)
print(f"   ✓ Loaded ratings: {len(ratings):,} rows")

In [ ]:
## Noise reduction

# Filter out wines with fewer than 5 ratings 
print("▶ Filtering wines with ≥5 ratings…")
wine_counts = ratings['WineID'].value_counts()
good_wines  = wine_counts[wine_counts >= 5].index
ratings     = ratings[ratings['WineID'].isin(good_wines)]
print(f"   ✓ Ratings remaining after wine filter: {len(ratings):,} rows")

# Filter out users with fewer than 5 reviews
print("▶ Filtering users with ≥5 reviews…")
user_counts = ratings['UserID'].value_counts()
good_users  = user_counts[user_counts >= 5].index
ratings     = ratings[ratings['UserID'].isin(good_users)]
print(f"   ✓ Ratings remaining after user filter: {len(ratings):,} rows")

# Merge ratings with wines metadata on 'WineID'
print("▶ Merging ratings with wines metadata…")
df = pd.merge(ratings, wines, on='WineID', how='inner')
print(f"   ✓ Merged DataFrame: {len(df):,} rows × {df.shape[1]:,} cols")

In [ ]:
# Time-based split of the dataset into train/val/test sets
# (80% train+val, 20% test)
print("▶ Splitting off test set (80/20 by user-time)…")
train_val, test = user_time_split(df, date_col='Date', split_ratio=0.8)
print(f"   ✓ train_val: {len(train_val):,} rows, test: {len(test):,} rows")
# (75% train, 25% val)
print("▶ Splitting train/val (75/25 by user-time)…")
train, val = user_time_split(train_val, date_col='Date', split_ratio=0.75)
print(f"   ✓ train: {len(train):,} rows, val: {len(val):,} rows")

In [ ]:

# Separate features (X) and target (y) for each set
# Save them as CSVs for later use
print("▶ Separating features (X) and target (y)…")
print("▶ Writing out CSVs…")
for name, split in [('train', train), ('val', val), ('test', test)]:
    X = split.drop(columns='Rating')
    y = split['Rating']
    X.to_csv(f'./data/X_{name}.csv', index=False)
    y.to_csv(f'./data/y_{name}.csv', index=False)
    print(f"   ✓ Wrote {name}: X_{name}.csv ({len(X):,} rows), y_{name}.csv")

In [ ]:
# Create profile report for the whole dataset, to get some insights on our data
profile = ProfileReport(df, title='XWines-merged Profiling Report')
profile.to_file('report.html')

## Now merged, formatted and splitted data can be loaded from the corresponding csv files

### Next steps:
* Define the preprocessing methods for different features:
    * **Standard Scaler** - is used for numerical type columns
        * **ABV**
        * **Vintage** (formatted to be numerical)
        * **DaysAgo(Date)** - see below
    * **One-hot-encoding** - is used for categorical features, but is limited by the number of categories within a feature:
        * **Type**
        * **Body**
        * **Acidity**
        * **Elaborate**
    * **Multi-label Binarizer** - is used for categorical features with too many categories, where also multiple active categories could be possible:
        * **Grapes** (774 classes)
        * **Harmonize** (~64 classes)
    * **Target Encoding** - used for text features and user IDs, wine IDs. Used with KFold(5 folds) to prevent data leakage, i.e. encoded feature never knows about it's own target value
    * **Date encoding** - created custom object to convert datetime column to DaysAgo from the most recent record column. This way we keep information about time-related information and reduce feature to be simply numerical. **Standard Scaler** applied afterwards.

In [ ]:
# Aggregate features

# Use StandardScaler for numerical features (create binary Is_NonVintage column derived from Vintage and maybe fill NaN values with 0)
numerical_features = ['ABV', 'Vintage']
# Use one-hot encoding for categorical features
categorical_features = ['Type', 'Elaborate', 'Body', 'Acidity']
# Use multilabel binarizer for multilabel features
multilabel_features = ['Grapes', 'Harmonize']
# Use target encoding for names and IDs
text_features = ['WineryName', 'RegionName', 'Country', 'UserID', 'WineID']
# Use conversion to DaysAgo for time-based features
date_features = ['Date']

* **Create Preprocessing pipeline**:
    ##### **Important**: Since during Grapes column encoding we create 774 classes + there are around 100 classes from other encoders, the pandas DataFrame would require too much RAM (I recieved 69GB memory allocation error only for the Grapes column) and same happening for the dense array (numpy), I used csr_matrix from scipy.sparse and some additional optimizations for MultiLabelBinarizer in particular, to be able to store all the preprocessed features. More info in code below.

    * **Created custom object for Date column preprocessing**
    * **Created Wrappers for other preprocessors to always return sparse csr matricies**. For OneHotEncoding there is already implemented sparse output. For StandardScaler in date_pipeline wrapper is not required, since the input is already a csr matrix.
    * **Created pipelines for each preprocessor and a general pipeline to combine everything together, using ColumnTransformer** 

In [ ]:
# Date preprocessor
class DateTransformer(BaseEstimator, TransformerMixin):
    """Transforms a single datetime column into 'days ago' relative to the latest date in training data."""
    
    def fit(self, X, y=None):
        # Expect a DataFrame with a single datetime column
        self.col = X.columns[0]
        self.column = f"{self.col}_days_ago"
        self.reference_date = pd.to_datetime(X[self.col]).max()
        return self

    def transform(self, X):
        days_ago = (self.reference_date - pd.to_datetime(X[self.col])).dt.days
        
        return csr_matrix(days_ago.values.reshape(-1, 1))

    def get_feature_names_out(self, input_features=None):
        return np.array([self.column])
    
class MultiLabelWrapper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoders = {}
        self.feature_names = []
    
    def fit(self, X, y=None):
        self.feature_names = []
        for col in X.columns:
            mlb = MultiLabelBinarizer()
            mlb.fit(X[col])
            self.encoders[col] = mlb
            # Store feature names for this column
            self.feature_names.extend([f"{col}__{cls}" for cls in mlb.classes_])
        return self
            
    def transform(self, X):
        matricies = []
        for col in X.columns:
            mlb = self.encoders[col]
            class_index = {cls: i for i, cls in enumerate(mlb.classes_)}
            n_rows = len(X)
            n_classes = len(mlb.classes_)
            sparse = lil_matrix((n_rows, n_classes), dtype=np.uint8)

            for i, labels in enumerate(X[col]):
                for label in labels:
                    if label in class_index:
                        sparse[i, class_index[label]] = 1

            matricies.append(sparse.tocsr())
        return hstack(matricies, format='csr')
    
    def get_feature_names_out(self, input_features=None):
        return np.array(self.feature_names)

class TargetEncoderWrapper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoders = {}
        self.feature_names = []
    
    def fit(self, X, y):
        self.feature_names = []
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        
        for col in X.columns:
            te = TargetEncoder(cols=[col])
            te.fit(X[[col]], y, cv=kf)
            self.encoders[col] = te
            # Store feature names for this column
            self.feature_names.append(f'{col}_target_encoded')
        return self
    
    def transform(self, X):
        matricies = []
        for col in X.columns:
            te = self.encoders[col]
            df_encoded = te.transform(X[[col]])
            arr = df_encoded[col].values.reshape(-1, 1)
            matricies.append(csr_matrix(arr))
        return hstack(matricies, format='csr')
    
    def get_feature_names_out(self, input_features=None):
        return np.array(self.feature_names)
    
class StandardScalerWrapper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler(with_mean=False)
        self.feature_names = []

    def fit(self, X, y=None):
        self.feature_names = X.columns.tolist()
        self.scaler.fit(X)
        return self

    def transform(self, X):
        X_scaled = self.scaler.transform(X)
        # Always return sparse csr_matrix
        if not issparse(X_scaled):
            X_scaled = csr_matrix(X_scaled)
        return X_scaled

    def get_feature_names_out(self, input_features=None):
        return np.array(self.feature_names)

## Preprocessing pipeline

# Numerical
numerical_pipeline = Pipeline([
    ('scaler', StandardScalerWrapper()),
    ])

# Categorical via one-hot-encoding
categorical_pipeline = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True))
])

# Categorical via multilabelbinarizer
multilabel_pipeline = Pipeline([
    ('multilabel', MultiLabelWrapper())
])

# Names and IDs via target encoding
text_pipeline = Pipeline([
    ('target', TargetEncoderWrapper())
])

# Datetime via custom date transformer
date_pipeline = Pipeline([
    ('date', DateTransformer()),
    ('scaler', StandardScaler(with_mean=False))
])

# Preprocessor
preprocessor = ColumnTransformer(transformers=[
    ('numerical', numerical_pipeline, numerical_features),
    ('categorical', categorical_pipeline, categorical_features),
    ('multilabel', multilabel_pipeline, multilabel_features),
    ('name_id', text_pipeline, text_features),
    ('date', date_pipeline, date_features)
])

preprocessing_pipeline = Pipeline([
    ('preprocessor', preprocessor)
])


* **Fit preprocessing pipeline on training data. We pass target variable there as well for the Target Encoder**
* **Transform X_train,X_val,_X_test on a fitted preprocessor**
* **Save results to .npz files. These are binary format files, which allow us to store our large sparse matrices. Implemented via save_npz() for scipy.sparse.**

In [ ]:
print("▶ Fitting preprocessing pipeline on training data…")
preprocessing_pipeline.fit(X_train, y_train)
print("   ✓ Pipeline fitted")

print("▶ Transforming X_train…")
X_train_preprocessed = preprocessing_pipeline.transform(X_train)
print(f"   ✓ X_train preprocessed: {X_train_preprocessed.shape[0]:,} rows × {X_train_preprocessed.shape[1]:,} features")

print("▶ Transforming X_val…")
X_val_preprocessed = preprocessing_pipeline.transform(X_val)
print(f"   ✓ X_val preprocessed: {X_val_preprocessed.shape[0]:,} rows × {X_val_preprocessed.shape[1]:,} features")

print("▶ Transforming X_test…")
X_test_preprocessed = preprocessing_pipeline.transform(X_test)
print(f"   ✓ X_test preprocessed: {X_test_preprocessed.shape[0]:,} rows × {X_test_preprocessed.shape[1]:,} features")


In [ ]:
# Save the processed data to NPZ files
print("▶ Writing out preprocessed data to NPZs…")
save_npz("X_train_preprocessed.npz", X_train_preprocessed)
print(f"   ✓ Wrote X_train_processed.npz ({X_train_preprocessed.shape[0]:,} rows × {X_train_preprocessed.shape[1]:,} features)")
save_npz("X_val_preprocessed.npz", X_val_preprocessed)
print(f"   ✓ Wrote X_val_processed.npz ({X_val_preprocessed.shape[0]:,} rows × {X_val_preprocessed.shape[1]:,} features)")
save_npz("X_test_preprocessed.npz", X_test_preprocessed)
print(f"   ✓ Wrote X_test_processed.npz ({X_test_preprocessed.shape[0]:,} rows × {X_test_preprocessed.shape[1]:,} features)")

* **Few test runs for different models without Hyperparameter optimization to confirm everything works.**
* **Do Hyperparameter optimization. Unfortunately we can't use Cross-Validation since our dataset is too large. After some research I decided to try optuna package which is quite easy to use. However, will need to see if it gives reasonable result in our case.**

#### Below is only sample for LightGBM model. I will add and run other models over the weekends and then we could benchmark the results.

In [ ]:
# Train a LightGBM model
lgb_model = lgb.LGBMRegressor(n_estimators=100, random_state=42)
lgb_model.fit(X_train_preprocessed, y_train)

y_pred = lgb_model.predict(X_test_preprocessed)
mse = mean_squared_error(y_test, y_pred)
print(f"Validation MSE: {mse:.4f}")

# I lost output for this cell, but it was around MSE: 0.37. Will rerun all cells again soon.

In [ ]:
def objective(trial):
    model = lgb.LGBMRegressor(
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3),
        num_leaves=trial.suggest_int('num_leaves', 31, 256),
        max_depth=trial.suggest_int('max_depth', -1, 20),
        min_child_samples=trial.suggest_int('min_child_samples', 5, 100),
        n_estimators=1000
    )

    model.fit(
        X_train_preprocessed, y_train,
        eval_set=[(X_val_preprocessed, y_val)],
        eval_metric='mse',
        callbacks=[lgb.early_stopping(50)],
    )

    y_pred = model.predict(X_val_preprocessed)
    mse = mean_squared_error(y_val, y_pred)
    return mse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=25)

print("✅ Best params:", study.best_params)
print("✅ Best MSE:", study.best_value)

In [ ]:
# Train a LightGBM model
lgb_model = lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.0503092382442247, num_leaves=169, max_depth=8, min_child_samples=82, random_state=42)
lgb_model.fit(X_train_preprocessed, y_train)

y_pred = lgb_model.predict(X_test_preprocessed)
mse = mean_squared_error(y_test, y_pred)
print(f"Validation MSE: {mse:.4f}")

# Result MSE: 0.3955, falls in a "good" range for target value in range 1 to 5
